In [1]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
    import torch
    from IPython.core.display import HTML
    from IPython.display import display 
    import os
    print("Running in Google Colab")
    if not torch.cuda.is_available():
        display(HTML("""<div style="background-color: red; font-weight: bold; color: white;">You have not activated a GPU in Google Colab. Follow the instructions in the <code style="color: white;">README</code></div>"""))
    print("Installing requirements")
    requirements_url = "https://raw.githubusercontent.com/willdalh/ml-course/main/requirements.txt"
    if not os.path.exists('requirements.txt'):
        !wget {requirements_url}
    %pip install --user -r requirements.txt

# Natural Language Processing (NLP)
Språkteknologi (NLP på engelsk) fikk en boost etter at [Transformer-arkitekturen](https://arxiv.org/abs/1706.03762) ble introdusert i 2017. Denne arkitekturen danner grunnlaget for de aller fleste tjenestene som kombinerer AI og språk idag, der ChatGPT er et godt eksempel. 

Siden da har [Hugging Face](huggingface.co) dukket opp som plattform som forsøker å tilgjengeliggjøre datasett, implementasjoner og erfaringer innen NLP. Av den grunn vil denne notebooken ta i bruk bibliotekene som tilbys fra dem. 

In [2]:
import torch

## Fyll inn det manglende ordet
Modellen som først demonstrerte styrken til Transformere, er BERT, som står for _Bidirectional Encoder Representations from Transformers_. Den lager altså vektorrepresentasjoner av ord ved å se på både ordene som kom før, samt de etterfølgende ordene i en setning. Nasjonalbiblioteket har trent en norsk versjon av denne, som vi finner på Hugging Face. 

Vi setter opp en unmasker som tar i bruk en BERT-modell i bunnen for å beregne hvilket ord som mangler. 

In [3]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='NbAiLab/nb-bert-base')

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
res = unmasker("I Norge brukes elektronisk [MASK] på sykehusene")
res[0]['token_str']

'journal'

## Sammenlikne setninger
BERT lager bare embeddinger av enkeltord, men i noen tilfeller ønsker vi å ha embeddinger for hele setninger, slik at vi kan sammenlikne dem. Til dette bruker vi en Sentence Transformer, og igjen har Nasjonalbiblioteket trent en som vi finner på Hugging Face.

In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('NbAiLab/nb-sbert-base')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.53k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
anchor = "Jeg liker å spise iskrem"
anchor_embedding = model.encode(anchor, convert_to_tensor=True)
print(anchor_embedding.shape)

torch.Size([768])


Vi ser at setningen er er blitt gjort om til en vektor med 768 tall.

In [7]:
sentences = ["Is er veldig godt", "Norske sykehus bruker elektronisk pasientjournal"]
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
print(sentence_embeddings.shape)

torch.Size([2, 768])


Vi skal nå sammenlikne embeddingene av de to setningene med anker-setningen. Embeddingene er vektorer/punkter, og det er flere måter å gjøre dette på. Vi bruker cosinus-likhet, som er cosinus av vinkelen som spennes mellom to vektorer. Men man kunne også brukt Euklidsk avstand. 

In [13]:
similarities_with_anchor = []
for emb in sentence_embeddings:
    cos_sim = util.cos_sim(anchor_embedding, emb).item()
    similarities_with_anchor.append(cos_sim)

print(f"Similarities with the sentence '{anchor}':")
for i, sentence in enumerate(sentences):
    print(f"{sentence}: {similarities_with_anchor[i]:.3f}")

Similarities with the sentence 'Jeg liker å spise iskrem':
Is er veldig godt: 0.669
Norske sykehus bruker elektronisk pasientjournal: 0.069


## Likhetssøk i database
Vi ser at embeddinger fungerer bra for å sammenlikne konteksten mellom to setninger, uten at ordene som brukes trenger å være helt like. Av denne grunn har det dukket opp flere tjenester som tilbyr å lagre embeddinger og utføre kjappe likhetssøk mellom dokumenter, der eksempler er [Redis Vector Database](https://redis.io/docs/get-started/vector-database/) og [Pinecone](https://www.pinecone.io/).

For å utforske hvordan vektordatabaser fungerer i praksis, lager vi en enkel implementasjon selv.

In [14]:
class VectorDatabase:
    def __init__(self, model: SentenceTransformer):
        self.documents = [] # list of strings
        self.embeddings = [] # list of torch.Tensors
        self.model = model # The SentenceTransformer model to use

    def add_document(self, document: str) -> None:
        """
        Add a document to the database. The document will be embedded using the model.

        Args:
            document (str): The document to add
        """
        self.documents.append(document)
        emb = self.model.encode(document, convert_to_tensor=True)
        self.embeddings.append(emb)

    def similarity_search(self, anchor_document: str, top_k: int = 3):
        """
        Search for the most similar documents to the anchor document.

        Args:
            anchor_document (str): The document to search for
            top_k (int, optional): The number of documents to return. Defaults to 3.

        Returns:
            list: A list of tuples containing the document and the similarity score
        """
        anchor_emb = self.model.encode(anchor_document, convert_to_tensor=True) # Embed the anchor document
        similarities = []
        for emb in self.embeddings: # Iterate all the document embeddings
            cos_sim = util.cos_sim(anchor_emb, emb).item()
            similarities.append(cos_sim)
        top_k_similar_indices = torch.topk(torch.tensor(similarities), k=top_k).indices.tolist() # Get the indices of the top k most similar documents
        res = [(self.documents[i], similarities[i]) for i in top_k_similar_indices] # Get the documents and their similarity scores
        return res
        

In [15]:
corpus = [ 
    "På norske sykehus brukes det elektroniske pasientjournaler", 
    "Bier er viktige for pollinering av mange av våre matvekster.",  
    "Mange nordmenn elsker å gå på ski om vinteren.",  
    "Astronomi er studiet av universet og dets himmellegemer.",
    "Ute i verdensrommet finnes det mange planeter og stjerner.",
    "Sjokolade er en populær godbit som lages fra kakaobønner.",  
    "Fotball er en av de mest populære sportene i verden.",  
    "Paris er kjent for sin arkitektur, mat og mote.",  
    "Elefanter er det største landdyret på jorden.",  
    "Bøker er en viktig kilde til kunnskap og underholdning.",  
    "Musikk er en universell form for kunst som uttrykker følelser og ideer.",  
    "Klimaendringer er en stor utfordring for verden i dag.",
    "Taco er vanlig å spise på fredager"
]  

vector_db = VectorDatabase(model)
for document in corpus:
    vector_db.add_document(document)

In [16]:
anchor_document = "Global oppvarming ødelegger jordkloden"
similar_documents = vector_db.similarity_search(anchor_document, top_k=1)
similar_documents

[('Klimaendringer er en stor utfordring for verden i dag.',
  0.5642092227935791)]

In [17]:
anchor_sentence = "Når det er mørkt og skyfritt kan man se stjerner"
similar_sentences = vector_db.similarity_search(anchor_sentence, top_k=3)
similar_sentences

[('Astronomi er studiet av universet og dets himmellegemer.',
  0.4471149444580078),
 ('Ute i verdensrommet finnes det mange planeter og stjerner.',
  0.4305839240550995),
 ('Musikk er en universell form for kunst som uttrykker følelser og ideer.',
  0.12972286343574524)]